In [ ]:
import os
import pandas as pd
import snowflake.connector

# Snowflake connection parameters
snowflake_config = {
    'user': 'your_username',
    'password': 'your_password',
    'account': 'your_account_url',
    'warehouse': 'your_warehouse',
    'database': 'your_database',
    'schema': 'your_schema',
}

# Function to create a Snowflake connection
def create_snowflake_connection():
    connection = snowflake.connector.connect(**snowflake_config)
    return connection

# Function to execute a Snowflake SQL statement
def execute_sql(connection, sql):
    cursor = connection.cursor()
    cursor.execute(sql)
    cursor.close()

# Function to load CSV data into Snowflake table
def load_csv_into_table(connection, table_name, csv_file_path):
    sql = f"""
    COPY INTO {table_name} FROM @%your_stage%
    FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER=1)
    """
    connection.cursor().execute(sql, _data=csv_file_path)

# Function to iterate over CSV files and load into Snowflake tables
def load_csv_files_into_snowflake():
    connection = create_snowflake_connection()

    # Assume you have a reference table called 'csv_reference' with columns 'table_name' and 'file_name'
    reference_sql = "SELECT table_name, file_name FROM csv_reference"

    reference_df = pd.read_sql(reference_sql, connection)

    for index, row in reference_df.iterrows():
        table_name = row['table_name']
        file_name = row['file_name']
        file_path = os.path.join('/path/to/your/csv/files', file_name)  # Update the path to your CSV files

        # Check if the CSV file exists
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue

        # Load CSV into Snowflake table
        load_csv_into_table(connection, table_name, file_path)
        print(f"Data loaded into table {table_name} from file {file_path}")

    connection.close()

if __name__ == "__main__":
    load_csv_files_into_snowflake()
